In [19]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
    
import keras
from keras import models
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model
from keras import backend as K
from keras import layers
from keras.layers import Layer
from keras.layers import Input,Dense,Flatten,Embedding,Permute,Dot,Reshape
from keras.layers.convolutional import Conv1D,MaxPooling1D,MaxPooling2D
from keras.layers import Dropout
from keras.layers import LSTM,GRU
from keras.callbacks import Callback
from keras.preprocessing import sequence
from keras.utils import np_utils
 
import tensorflow as tf
from tensorflow.python.framework import function
import math
 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.ensemble import RandomForestRegressor

import lightgbm as lgb
import xgboost as xgb 

from bs4 import BeautifulSoup
    
import scipy
import statsmodels.api
import statsmodels as sm
 
import copy
import random
import time
from time import sleep

import re
import os
import lightgbm as lgb

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

import spacy

from spellchecker import SpellChecker
import re
import nltk

# 1. 전처리 함수 및 Tokenizer 정의

In [13]:
sp = spacy.load('en_core_web_sm')

In [15]:
def making_train_df(data):

    encoding_result = DataFrame()

    for i in data["essay_id"].unique():
        text = data.loc[data["essay_id"] == i, "essay"]

        pos_tagging = list(map(lambda x : 
                           [x.lemma_], 
                           sp(text.values[0])))

        result_text = DataFrame(columns = ["lemma"])

        for j in range(0, len(pos_tagging)):
            result_text = pd.concat([result_text,
                                          DataFrame(pos_tagging[j], 
                                                    columns = ["lemma"])])

        morphs = list(result_text["lemma"])

        morphsVectored = list()

        ## 다시 형태소 분석기를 돌려서, text의 문장을 형태소로 분해하여 morphs에 담는다.
        ## 시간이 오래 걸립니다. 5분정도

        for j in morphs:
            temporailyList = list()

            try:
                # 색인사전에 있는 단어면 인덱스를 반환
                temporailyList.append(vocabulary[j])
            except KeyError:
                # 없는 단어면 0으로 대체한다.
                temporailyList.append(0)
            morphsVectored.append(temporailyList)

        vector_shell_text = np.array([0 for i in range(0, np.max(list(vocabulary.values())) + 1)])

        vector_shell_text[np.reshape(np.array(morphsVectored),[-1])] = 1

        encoding_result = pd.concat([encoding_result, DataFrame(vector_shell_text).T])


    train_df = pd.concat([data, encoding_result.reset_index(drop = True)], axis = 1)

    train_df.columns = train_df.columns.astype("str")
    
    return train_df

In [49]:
def making_ratio(data, score):

    pos_tagging = [list(map(lambda x : 
                       [data.loc[i, "essay_id"], x.text, x.tag_, x.lemma_], 
                       sp(data["essay"][i]))) 
              for i in range(0, len(data))]

    result = DataFrame(columns = ["essay_id","word","tag","lemma"])

    for i in range(0, len(pos_tagging)):
        result = pd.concat([result, DataFrame(pos_tagging[i], columns = ["essay_id","word","tag","lemma"])])
    
    sen_count = data["essay"].apply(lambda x : len(nltk.sent_tokenize(x))).reset_index(name = "sen_count")

    sen_count = pd.concat([data["essay_id"], data["essay"].apply(lambda x : len(nltk.sent_tokenize(x)))], axis = 1)

    sen_count.columns = ["essay_id","sen_count"]

    len_count = result.groupby("essay_id")["lemma"].count().reset_index()

    len_count.columns = ["essay_id","len_count"]

    ## 각 품사를 총 단어 갯수로 나눔

    count_result = DataFrame({"essay_id" : result["essay_id"].unique()})
    
    for i in result["tag"].unique():

        test = result[result["tag"] == i]

        lemma_count = test.groupby("essay_id")["lemma"].count().reset_index()

        count = pd.merge(lemma_count, len_count)

        count["{}_ratio".format(i)] = count["lemma"] / count["len_count"]

        count_result = pd.merge(count_result, count, how = "left")

    count_result = count_result.fillna(0)

    count_result = pd.merge(count_result, data[["essay_id",score]])
    
    return count_result

In [55]:
def making_cos_simil(data, criterion):


    pos_tagging = list(map(lambda x : 
                           [x.lemma_], 
                           sp(criterion)))

    result_criterion = DataFrame(columns = ["lemma"])

    for i in range(0, len(pos_tagging)):
        result_criterion = pd.concat([result_criterion, 
                                      DataFrame(pos_tagging[i], 
                                                columns = ["lemma"])])

    morphs = list(result_criterion["lemma"])

    morphsVectored = list()

    ## 다시 형태소 분석기를 돌려서, text의 문장을 형태소로 분해하여 morphs에 담는다.

    for i in morphs:
        temporailyList = list()
        #print(k)
        try:
            # 색인사전에 있는 단어면 인덱스를 반환
            temporailyList.append(vocabulary[i])
        except KeyError:
            # 없는 단어면 0으로 대체한다.
            temporailyList.append(0)
        morphsVectored.append(temporailyList)


    vector_shell = np.array([0 for i in range(0, np.max(list(vocabulary.values())) + 1)])

    vector_shell[np.reshape(np.array(morphsVectored),[-1])] = 1

    cos_simil = list()

    for i in data["essay_id"].unique():
        text = data.loc[data["essay_id"] == i, "essay"]

        pos_tagging = list(map(lambda x : 
                           [x.lemma_], 
                           sp(text.values[0])))

        result_text = DataFrame(columns = ["lemma"])

        for j in range(0, len(pos_tagging)):
            result_text = pd.concat([result_text,
                                          DataFrame(pos_tagging[j], 
                                                    columns = ["lemma"])])

        morphs = list(result_text["lemma"])

        morphsVectored = list()

        ## 다시 형태소 분석기를 돌려서, text의 문장을 형태소로 분해하여 morphs에 담는다.
        ## 시간이 오래 걸립니다. 5분정도

        for j in morphs:
            temporailyList = list()
            #print(k)
            try:
                # 색인사전에 있는 단어면 인덱스를 반환
                temporailyList.append(vocabulary[j])
            except KeyError:
                # 없는 단어면 0으로 대체한다.
                temporailyList.append(0)
            morphsVectored.append(temporailyList)

        vector_shell_text = np.array([0 for i in range(0, np.max(list(vocabulary.values())) + 1)])

        vector_shell_text[np.reshape(np.array(morphsVectored),[-1])] = 1

        simil_result = vector_shell.dot(vector_shell_text) / (np.linalg.norm(vector_shell) * np.linalg.norm(vector_shell_text))

        cos_simil.append(simil_result)

        
    return cos_simil   

# 2. Data Load 및 준비

In [2]:
essay = pd.read_csv("data/essay_dataset.csv")

In [3]:
# @정규표현식

searcher = re.compile('@')

In [4]:
essay["essay"] = essay["essay"].apply(lambda x : searcher.sub('',x))

In [5]:
def vect_tokenizer(text):
    return nltk.tokenize.word_tokenize(text)

In [6]:
text = essay["essay"].copy()

In [7]:
text = text.dropna()

In [8]:
## CountVectorizer로 문자 : 숫자로 이루어진 색인 사전을 만든다.
## 이미 만들어진 색인사전을 이용해도 됩니다.

vect = CountVectorizer(tokenizer = vect_tokenizer ,min_df = 1, analyzer = "word")
vect.fit(text)
vocabulary = vect.vocabulary_
# CLS와 EOS 토큰을 각각 정의한다.
vocabulary['CLS'] = len(vocabulary) + 1
vocabulary['EOS'] = len(vocabulary) + 2
DataFrame([vect.vocabulary_]).to_csv("data/색인사전.csv",encoding="utf-8",index=False)

C:\Users\User\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


# 3. 데이터 전처리 및 학습, 평가

## (1) Essay_1

In [9]:
essay_1 = essay[essay["essay_set"] == 1]

essay_1["sen_len"] = essay_1["essay"].apply(lambda x : len(x))

<ipython-input-9-4bc8cbc26f67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_1["sen_len"] = essay_1["essay"].apply(lambda x : len(x))


In [17]:
train_df_1 = making_train_df(essay_1)

In [22]:
# sen_len 포함

columns = [i for i in range(0, np.max(list(vocabulary.values())) + 1)]

columns.extend(["sen_len"])

columns = np.array(columns)

train_x, val_x, train_y, val_y = train_test_split(train_df_1[columns.astype("str")], 
                                                  train_df_1["score"],
                                                 random_state = 42)

rf_1 = RandomForestRegressor(n_jobs=-1, random_state = 42)

rf_1.fit(train_x, train_y)

predicted = np.round(rf_1.predict(val_x))

sklearn.metrics.mean_absolute_error(val_y, predicted)

0.5403587443946188

## (2) Essay_2

In [116]:
train_df_2 = making_train_df(essay_2)

In [51]:
essay_2 = essay[essay["essay_set"] == 2].reset_index(drop = True)

essay_2["sen_len"] = essay_2["essay"].apply(lambda x : len(x))

essay_2_ratio = making_ratio(essay_2, "score")

In [101]:
pos_tagging = [list(map(lambda x : 
                       [essay_2.loc[i, "essay_id"], x.text, x.pos_, x.tag_, x.lemma_, x.dep_], 
                       sp(essay_2["essay"][i]))) 
              for i in range(0, len(essay_2))]

result = DataFrame(columns = ["essay_id","word","pos","tag","lemma","depend"])

for i in range(0, len(pos_tagging)):
    result = pd.concat([result, DataFrame(pos_tagging[i], columns = ["essay_id","word","pos","tag","lemma","depend"])])

In [133]:
sen_count = essay_2["essay"].apply(lambda x : len(nltk.sent_tokenize(x))).reset_index(name = "sen_count")

sen_count = pd.concat([essay_2["essay_id"], essay_2["essay"].apply(lambda x : len(nltk.sent_tokenize(x)))], axis = 1)

sen_count.columns = ["essay_id","sen_count"]

In [119]:
len_count = result.groupby("essay_id")["lemma"].count().reset_index()

len_count.columns = ["essay_id","len_count"]

In [135]:
result = result.reset_index(drop = True)

lemma_reason = DataFrame({"pos" : ["SCONJ"]})

test = pd.merge(lemma_reason, result, how = "left")

lemma_count = test.groupby("essay_id")["lemma"].count().reset_index()

In [136]:
count = pd.merge(lemma_count, sen_count)

count["SCONJ_ratio"] = count["lemma"] / count["sen_count"]

count = pd.merge(count, essay_2[["essay_id","score"]])

count = count[["essay_id","lemma","sen_count","SCONJ_ratio","score"]]

In [141]:
result = result.reset_index(drop = True)

lemma_reason = DataFrame({"lemma" : ["if","since","when","as","because","unless"]})

test = pd.merge(lemma_reason, result, how = "left")

lemma_count = test.groupby("essay_id")["lemma"].count().reset_index()

In [142]:
count_2 = pd.merge(lemma_count, sen_count)

count_2["lemma_sen_ratio"] = count_2["lemma"] / count_2["sen_count"]

count_2 = pd.merge(count_2, essay_2[["essay_id","score"]])

count_2 = count_2[["essay_id","lemma","sen_count","lemma_sen_ratio","score"]]

In [125]:
train_df_2 = pd.merge(train_df_2,essay_2_ratio[["essay_id","JJ_ratio"]])

In [138]:
train_df_2 = pd.merge(train_df_2, count[["essay_id","SCONJ_ratio"]])

In [144]:
train_df_2 = pd.merge(train_df_2, count_2[["essay_id","lemma_sen_ratio"]])

In [145]:
# sen_len, JJ_ratio포함

columns = [i for i in range(0, 39755)]

columns.extend(["sen_len","JJ_ratio"])

columns = np.array(columns)

train_x, val_x, train_y, val_y = train_test_split(train_df_2[columns.astype("str")], 
                                                  train_df_2["score"],
                                                 random_state = 42)

rf = RandomForestRegressor(n_jobs=-1, random_state = 42)

rf.fit(train_x, train_y)

predicted = np.round(rf.predict(val_x))

sklearn.metrics.mean_absolute_error(val_y, predicted)

0.30180180180180183

## (3) Essay_3

In [71]:
essay_3 = essay[essay["essay_set"] == 3].reset_index(drop = True)

essay_3["sen_len"] = essay_3["essay"].apply(lambda x : len(x))

In [76]:
criterion = 'The response should describe a mood of gratitude, love, similar appreciative mood. The response may include, but is not limited to: the author says he is “eternally grateful” to his parents for instilling in him a love of cooking. He also credits them for his appreciation of Cuban music "which I adore to this day." In general he notes their having made an inviting home filled with "endless celebrations" out of "modest" means. the author credits his parents for instilling in him a great sense of "family" due to the "environment" they created. This sense of family extended to everyone in a time when the larger world was uninviting.the author mentions his family’s generosity in allowing others to stay with them and notes its reciprocal nature the author recognizes that his parents came to America "selflessly" in order to "give their children a better life." He details their challenges and obstacles and observes that they "endured." "I will always be grateful to my parents for their love and sacrifice. I’ve often told them that what they did was a much morecourageous thing than I could have ever done." He mentions his admiration andhaving thanked them yet admits that he has, "no way to express my gratitude." "I will never forget that house or its gracious neighborhood or the many things I learned there about how to love. I will never forget how my parents turned this simple house into a home"'

cos_simil = making_cos_simil(essay_3, criterion)

essay_3["cos_simil"] = cos_simil

train_df_3 = making_train_df(essay_3)

In [78]:
# sen_len, JJ_ratio포함

columns = [i for i in range(0, 39755)]

columns.extend(["sen_len"])

columns = np.array(columns)

train_x, val_x, train_y, val_y = train_test_split(train_df_2[columns.astype("str")], 
                                                  train_df_2["score"],
                                                 random_state = 42)

rf = RandomForestRegressor(n_jobs=-1, random_state = 42)

rf.fit(train_x, train_y)

predicted = np.round(rf.predict(val_x))

sklearn.metrics.mean_absolute_error(val_y, predicted)

0.32666666666666666

## (4) Essay_4

In [79]:
essay_4 = essay[essay["essay_set"] == 4].reset_index(drop = True)

In [80]:
essay_4["sen_len"] = essay_4["essay"].apply(lambda x : len(x))

In [81]:
criterion = 'The obstacles to dirigible docking include Building a mast on top of the building Meeting with engineers and dirigible engineers Transmitting the stress of the dirigible all the way down the building; the frame had to be shored up to the tune of $60,000 Housing the winches and other docking equipment Dealing with flammable gases Handling the violent air currents at the top of the building Confronting laws banning airships from the area Getting close enough to the building without puncturing'

cos_simil = making_cos_simil(essay_4, criterion)

essay_4["cos_simil"] = cos_simil

train_df_4 = making_train_df(essay_4)

In [83]:
# cos_simil, sen_len 포함

columns = [i for i in range(0, 39755)]

columns.extend(["cos_simil", "sen_len"])

columns = np.array(columns)

train_x, val_x, train_y, val_y = train_test_split(train_df_4[columns.astype("str")], 
                                                  train_df_4["score"],
                                                 random_state = 42)

rf = RandomForestRegressor(n_jobs=-1, random_state = 42)

rf.fit(train_x, train_y)

predicted = np.round(rf.predict(val_x))

sklearn.metrics.mean_absolute_error(val_y, predicted)

0.33555555555555555

## (5) Essay_5

In [84]:
essay_5 = essay[essay["essay_set"] == 5].reset_index(drop = True)

essay_5["sen_len"] = essay_5["essay"].apply(lambda x : len(x))

In [85]:
criterion = 'Being Patient Means that you are understanding and tolerant. A patient Person experiences difficulties without complaining.'

cos_simil = making_cos_simil(essay_5, criterion)

essay_5["cos_simil"] = cos_simil

train_df_5 = making_train_df(essay_5)

In [86]:
essay_5_ratio = making_ratio(essay_5, "score")

In [90]:
train_df_5 = pd.merge(train_df_5, essay_5_ratio[["essay_id","NN_ratio"]])

In [92]:
# sen_len, nn_ratio 포함(Best Score)

columns = [i for i in range(0, 39755)]

columns.extend(["sen_len", "NN_ratio"])

columns = np.array(columns)

train_x, val_x, train_y, val_y = train_test_split(train_df_5[columns.astype("str")], 
                                                  train_df_5["score"],
                                                 random_state = 42)

rf = RandomForestRegressor(n_jobs=-1, random_state = 42)

rf.fit(train_x, train_y)

predicted = np.round(rf.predict(val_x))

sklearn.metrics.mean_absolute_error(val_y, predicted)

2.3256997455470736

## (6) Essay_6

In [93]:
essay_6 = essay[essay["essay_set"] == 6].reset_index(drop = True)

essay_6["sen_len"] = essay_6["essay"].apply(lambda x : len(x))

In [94]:
criterion = '"If you want a place in the sun, you will have to expect some blisters." Tell a true story about a time when this quote was true for you or someone you know.'

cos_simil = making_cos_simil(essay_6, criterion)

essay_6["cos_simil"] = cos_simil

train_df_6 = making_train_df(essay_6)

In [95]:
# sen_len, cos_simil,  포함

columns = [i for i in range(0, 39755)]

columns.extend(["sen_len", "cos_simil"])

columns = np.array(columns)

train_x, val_x, train_y, val_y = train_test_split(train_df_6[columns.astype("str")], 
                                                  train_df_6["score"],
                                                 random_state = 42)

rf = RandomForestRegressor(n_jobs=-1, random_state = 42)

rf.fit(train_x, train_y)

predicted = np.round(rf.predict(val_x))

sklearn.metrics.mean_absolute_error(val_y, predicted)

3.2154696132596685